In [1]:
import os
import tqdm
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, StratifiedKFold

In [2]:
from MMFE import MMFE, normalize, denormalize, get_metrics

In [3]:
from sklearn.linear_model import ElasticNet, Ridge, Lasso
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
import xgboost as xgb

In [4]:
random_state = 2025
np.random.seed(random_state)
torch.manual_seed(random_state)
if torch.cuda.is_available():
    torch.cuda.manual_seed(random_state)
    torch.cuda.manual_seed_all(random_state)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [5]:
sns.set_theme(style='whitegrid')

# 1. Initialization

## Task

In [6]:
TASKNAME = '300nM'

output_dir = os.path.join('results', TASKNAME)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

output_dir = os.path.join(output_dir, 'MMFE')
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

## Torch Device

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Data Load

In [8]:
df_raw = pd.read_csv(os.path.join('data', 'selectivity.csv'))

In [9]:
df_raw

,Compound,Drug name,PubChem CID,Binding Mode (based on ABL1-phos. vs. -nonphos affinity),S(300nM),S(3uM),SMILES
0,A-674563,NaN,11314340,undetermined,0.1166,0.2772,CC1=C2C=C(C=CC2=NN1)C3=CC(=CN=C3)OC[C@H](CC4=C...
1,AB-1010,Masitinib,10074640,Type II,0.0337,0.0622,CC1=C(C=C(C=C1)NC(=O)C2=CC=C(C=C2)CN3CCN(CC3)C...
2,ABT-869,Linifanib,11485656,undetermined,0.0648,0.1839,CC1=CC(=C(C=C1)F)NC(=O)NC2=CC=C(C=C2)C3=C4C(=C...
3,AC220,Quizartinib,24889392,Type II,0.0285,0.0751,CC(C)(C)C1=CC(=NO1)NC(=O)NC2=CC=C(C=C2)C3=CN4C...
4,AG-013736,Axitinib,6450551,Type I,0.0570,0.1969,CNC(=O)C1=CC=CC=C1SC2=CC3=C(C=C2)C(=NN3)/C=C/C...
...,...,...,...,...,...,...,...
67,TG-100-115,NaN,10427712,Type I,0.0337,0.1321,C1=CC(=CC(=C1)O)C2=NC3=C(N=C(N=C3N=C2C4=CC(=CC...
68,TG-101348,NaN,16722836,Type I,0.1788,0.5389,CC1=CN=C(N=C1NC2=CC(=CC=C2)S(=O)(=O)NC(C)(C)C)...
69,Vandetanib,NaN,3081361,Type I,0.0933,0.2358,CN1CCC(CC1)COC2=C(C=C3C(=C2)N=CN=C3NC4=C(C=C(C...
70,VX-680/MK-0457,Tozasertib,5494449,Type I,0.1321,0.3472,CC1=CC(=NN1)NC2=CC(=NC(=N2)SC3=CC=C(C=C3)NC(=O...


In [10]:
X_raw = df_raw['SMILES'].values
y_raw = df_raw[f'S({TASKNAME})'].values.astype(np.float32)

# 2. 5-Fold Cross-Validation

In [11]:
# 5-fold cross-validation setup
y_binned = pd.qcut(y_raw, q=5, labels=False, duplicates='drop')
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_state)

ys = []

for fold, (train_idx, test_idx) in enumerate(tqdm.tqdm(skf.split(X_raw, y_binned), total=5, desc="5-Fold CV")):
    ###################################################################
    ## Data Splits
    ###################################################################
    # Test set
    X_te = X_raw[test_idx]
    y_te = y_raw[test_idx]
    
    # Train set (further split into train and validation)
    X_train_full = X_raw[train_idx]
    y_train_full = y_raw[train_idx]
    
    # Split train into train and validation
    idx_tr, idx_va = train_test_split(range(len(X_train_full)), test_size=0.1, random_state=2025)
    
    X_tr = X_train_full[idx_tr]
    y_tr = y_train_full[idx_tr]
    X_va = X_train_full[idx_va]
    y_va = y_train_full[idx_va]

    ###################################################################
    ## Model Initialization
    ###################################################################
    model = MMFE(output_dir, device, fold)

    ###################################################################
    ## Model Training
    ###################################################################
    _ = model.fit(X_tr, X_va, y_tr.reshape(-1,1), y_va.reshape(-1,1), temperature=1.0)

    ###################################################################
    ## Embeddings
    ###################################################################
    H_tv = model.predict(np.hstack([X_tr, X_va]))
    z_tv = normalize(np.hstack([y_tr, y_va]))
    H_te = model.predict(X_te)

    ###################################################################
    ## Classifier
    ###################################################################
    clfs = {
        'ElasticNet': ElasticNet(alpha=0.01, random_state=random_state),
        'Ridge': Ridge(random_state=random_state),
        'Lasso': Lasso(alpha=0.01, random_state=random_state),
        'SVR': SVR(),
        'KNN': KNeighborsRegressor(),
        'DecisionTree': DecisionTreeRegressor(random_state=random_state),
        'RandomForest': RandomForestRegressor(random_state=random_state),
        'AdaBoost': AdaBoostRegressor(random_state=random_state),
        'XGBoost': xgb.XGBRegressor(random_state=random_state)
    }

    # Process each test sample for this fold
    fold_results = []
    for i, (gt, h_te) in enumerate(zip(y_te, H_te)):
        p_te = {'GroundTruth': gt, 'Fold': fold}
        for clf_name, clf in clfs.items():
            clf.fit(H_tv, z_tv)
            p_te[clf_name] = denormalize(clf.predict(h_te.reshape(1, -1))[0])
        fold_results.append(p_te)
        ys.append(p_te)
    
    ###################################################################
    ## Save individual fold results
    ###################################################################
    # Create fold directory
    fold_dir = os.path.join(output_dir, f'Fold{fold+1}')
    if not os.path.exists(fold_dir):
        os.makedirs(fold_dir)
    
    # Save fold predictions
    df_fold = pd.DataFrame(fold_results)
    df_fold.to_csv(os.path.join(fold_dir, 'predictions.csv'), index=False)
    
    # Calculate and save fold metrics
    fold_metrics = []
    model_names = [col for col in df_fold.columns if col not in ['GroundTruth', 'Fold']]
    
    for model_name in model_names:
        fold_res = get_metrics(df_fold['GroundTruth'], df_fold[model_name])
        fold_res['model_name'] = model_name
        fold_metrics.append(fold_res)
    
    df_fold_metrics = pd.DataFrame(fold_metrics)
    df_fold_metrics.to_csv(os.path.join(fold_dir, 'metrics.csv'), index=False)

5-Fold CV:   0%|          | 0/5 [00:00<?, ?it/s]/home/rlawlsgurjh/hdd/miniconda3/envs/mmfe/lib/python3.9/site-packages/torch/functional.py:534: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /opt/conda/conda-bld/pytorch_1729647350383/work/aten/src/ATen/native/TensorShape.cpp:3595.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
5-Fold CV: 100%|██████████| 5/5 [10:17<00:00, 123.52s/it]


In [12]:
df_ys = pd.DataFrame(ys)

In [13]:
df_ys

,GroundTruth,Fold,ElasticNet,Ridge,Lasso,SVR,KNN,DecisionTree,RandomForest,AdaBoost,XGBoost
0,0.0337,0,0.028858,0.028572,0.029775,0.025506,0.027178,0.0389,0.027852,0.030699,0.024150
1,0.2617,0,0.080228,0.078321,0.079738,0.094777,0.049093,0.0674,0.089338,0.077700,0.094734
2,0.0803,0,0.046658,0.046698,0.047241,0.047903,0.046177,0.0285,0.040657,0.042954,0.052514
3,0.0518,0,0.118870,0.123154,0.121780,0.129449,0.165144,0.0285,0.115802,0.109592,0.066784
4,0.0285,0,0.038618,0.039270,0.036464,0.038306,0.032352,0.0233,0.029125,0.039500,0.038691
...,...,...,...,...,...,...,...,...,...,...,...
67,0.0544,4,0.114262,0.107510,0.122373,0.112649,0.106891,0.1166,0.126491,0.102571,0.085751
68,0.0311,4,0.062382,0.063136,0.065268,0.074131,0.056500,0.0674,0.083257,0.056684,0.071389
69,0.3420,4,0.091320,0.090207,0.093577,0.098607,0.096380,0.0674,0.093668,0.068970,0.054912
70,0.3109,4,0.216854,0.213630,0.218411,0.215218,0.166695,0.1813,0.199033,0.211873,0.230521


In [14]:
results = []
model_names = [col for col in df_ys.columns if col not in ['GroundTruth', 'Fold']]

for model_name in model_names:
    fold_metrics = []
    for fold in range(5):
        fold_data = df_ys[df_ys['Fold'] == fold]
        if len(fold_data) > 0:
            fold_res = get_metrics(fold_data['GroundTruth'], fold_data[model_name])
            fold_metrics.append(fold_res)
    
    # Calculate mean and std across folds
    if fold_metrics:
        rmse_values = [m['rmse'] for m in fold_metrics]
        r2_values = [m['r2'] for m in fold_metrics]
        pcc_values = [m['pcc'] for m in fold_metrics]
        
        res = {
            'model_name': model_name,
            'rmse': round(np.mean(rmse_values), 3),
            'rmse_std': round(np.std(rmse_values), 3),
            'r2': round(np.mean(r2_values), 3),
            'r2_std': round(np.std(r2_values), 3),
            'pcc': round(np.mean(pcc_values), 3),
            'pcc_std': round(np.std(pcc_values), 3)
        }
        results.append(res)

df_res = pd.DataFrame(results)

In [15]:
df_res.to_csv(os.path.join(output_dir, 'metrics.csv'), index=False)

In [16]:
df_res

,model_name,rmse,rmse_std,r2,r2_std,pcc,pcc_std
0,ElasticNet,0.112,0.021,-0.063,0.528,0.412,0.357
1,Ridge,0.112,0.021,-0.057,0.524,0.416,0.351
2,Lasso,0.113,0.022,-0.072,0.538,0.407,0.359
3,SVR,0.113,0.020,-0.042,0.405,0.379,0.350
4,KNN,0.115,0.017,-0.125,0.541,0.349,0.393
5,DecisionTree,0.125,0.030,-0.261,0.574,0.229,0.394
6,RandomForest,0.119,0.017,-0.225,0.661,0.363,0.368
7,AdaBoost,0.119,0.017,-0.173,0.471,0.320,0.357
8,XGBoost,0.118,0.021,-0.237,0.712,0.311,0.424
